# Download CIFAR-10 Dataset

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

==> Preparing data..


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
from __future__ import print_function
import torch
import time

# Define class for residual block that accepts module
class ResidualBlock(torch.nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, inputs):
        return self.module(inputs) + inputs


model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, kernel_size=7), # 32x32 -> 26x26
    torch.nn.LeakyReLU(),
    ResidualBlock(
        torch.nn.Sequential(
            torch.nn.Conv2d(32, 32, kernel_size=5, padding=2),
            torch.nn.LeakyReLU(),
            torch.nn.Conv2d(32, 32, kernel_size=3, padding=1),
            torch.nn.LeakyReLU(),
        )
    ),
    torch.nn.BatchNorm2d(32),
    torch.nn.Conv2d(32, 64, kernel_size=7), # 26x26 -> 20x20
    torch.nn.LeakyReLU(),
    torch.nn.MaxPool2d(2, 2), # 20x20 -> 10x10
    ResidualBlock(
        torch.nn.Sequential(
            torch.nn.Conv2d(64, 64, kernel_size=5, padding=2),
            torch.nn.LeakyReLU(),
            torch.nn.Conv2d(64, 64, kernel_size=3, padding=1),
            torch.nn.LeakyReLU(),
        )
    ),
    torch.nn.BatchNorm2d(64),
    torch.nn.Conv2d(64, 128, kernel_size=10), # 10x10 -> 1x1
    torch.nn.LeakyReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(128, 32),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(32, 10)
)

def train(model, device, train_loader, optimizer, epoch):
    model.train() # set the model into training model (evaluation model in the testing)
    count = 0
    cross_entropy = torch.nn.CrossEntropyLoss()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        loss = cross_entropy(model(data), target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, min(batch_idx * train_loader.batch_size, len(train_loader.dataset)), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += torch.nn.functional.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

time0 = time.time()
no_cuda = False
save_model = False
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(100)
device = torch.device("cuda" if use_cuda else "cpu")

model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

epochs = 100
for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    loss_val = test( model, device, test_loader)
    scheduler.step(loss_val)

if (save_model):
    torch.save(model.state_dict(), "cifar_lenet.pt")

time1 = time.time()
print ('Traning and Testing total excution time is: %s seconds ' % (time1 - time0))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.303519
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.228994
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.087996
Train Epoch: 1 [3840/50000 (8%)]	Loss: 1.819777
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.760368
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.724587
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.749140
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.712685
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.729418
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.505343
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.620837
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.584856
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.878262
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.572523
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.588883
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.535163
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.815227
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.644291
Train Epoch: 1 [23040/50000 (46%)]	Loss: 1.716015
Train Epoch: 1 [24320/50000 (49%)]	Loss: 1.518115
Train Epoch: 1 